# Sharing_Variables

googlesource.com [blog](https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.7/tensorflow/g3doc/how_tos/variable_scope/index.md)

# Fail to share variables

In [1]:
import tensorflow as tf

def my_op(x):
    W = tf.Variable(tf.random_normal([32, 10]), name="W")
    b = tf.Variable(tf.zeros([10]), name="b")
    return x @ W + b

image1 = tf.placeholder(tf.float32, shape=(None, 32), name='image1')
image2 = tf.placeholder(tf.float32, shape=(None, 32), name='image2')

# First call creates one set of variables.
# <tf.Variable 'W:0' shape=(32, 10) dtype=float32_ref>
# <tf.Variable 'b:0' shape=(10,) dtype=float32_ref>
result1 = my_op(image1)

# Another set is created in the second call.
# <tf.Variable 'W_1:0' shape=(32, 10) dtype=float32_ref>
# <tf.Variable 'b_1:0' shape=(10,) dtype=float32_ref>
result2 = my_op(image2)

print(tf.GraphKeys.TRAINABLE_VARIABLES)
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

/Users/sungchul/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/sungchul/anaconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


trainable_variables
[<tf.Variable 'W:0' shape=(32, 10) dtype=float32_ref>, <tf.Variable 'b:0' shape=(10,) dtype=float32_ref>, <tf.Variable 'W_1:0' shape=(32, 10) dtype=float32_ref>, <tf.Variable 'b_1:0' shape=(10,) dtype=float32_ref>]


# First Solution - variables_dict

A common way to share variables is to create them in a separate piece of code and pass them to functions that use them. 

While convenient, creating variables like above, outside of the code, breaks encapsulation:

- The code that builds the graph must document the names, types, and shapes of variables to create.
- When the code changes, the callers may have to create more, or less, or different variables.

https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.7/tensorflow/g3doc/how_tos/variable_scope/index.md

In [1]:
import tensorflow as tf

variables_dict = {"weight": tf.Variable(tf.random_normal([32, 10]), name="W"), \
                  "bias": tf.Variable(tf.zeros([10]), name="b")}

def my_op(x, variables_dict):
    return x @ variables_dict['weight'] + variables_dict['bias']

image1 = tf.placeholder(tf.float32, shape=(None, 32), name='image1')
image2 = tf.placeholder(tf.float32, shape=(None, 32), name='image2')

# First call creates one set of variables.
# <tf.Variable 'W:0' shape=(32, 10) dtype=float32_ref>
# <tf.Variable 'b:0' shape=(10,) dtype=float32_ref>
result1 = my_op(image1, variables_dict)

# The same variables set is reused in the second call.
# <tf.Variable 'W:0' shape=(32, 10) dtype=float32_ref>
# <tf.Variable 'b:0' shape=(10,) dtype=float32_ref>
result2 = my_op(image2, variables_dict)

print(tf.GraphKeys.TRAINABLE_VARIABLES)
print(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

/Users/sungchul/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/sungchul/anaconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


trainable_variables
[<tf.Variable 'W:0' shape=(32, 10) dtype=float32_ref>, <tf.Variable 'b:0' shape=(10,) dtype=float32_ref>]


# Second Solution - tf.get_variable

tf.get_variable either shares or creates a new one:

- Case 1: If there is no variable with given name (including prefix) used yet, it creates a new one.

- Case 2: If there is a variable with given name (including prefix) used before and if we declare to reuse, it uses or shares the old one. 

- Case 3: If there is a variable with given name (including prefix) used before but if we don't declare to reuse, it raises ValueError. 

In [ ]:
import tensorflow as tf

if 1:
    # ValueError: Variable v already exists, disallowed. 
    # Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? 
    a = tf.get_variable("v", shape=[1]) 
    b = tf.get_variable("v", shape=[1])
elif 1:
    # TypeError: get_variable() got an unexpected keyword argument 'reuse'
    a = tf.get_variable("v", shape=[1]) 
    b = tf.get_variable("v", shape=[1], reuse=True)
elif 1:
    # ValueError: Variable one/v already exists, disallowed. 
    # Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? 
    with tf.variable_scope("one"):
        a = tf.get_variable("v", shape=[1]) 
    with tf.variable_scope("one"):
        b = tf.get_variable("v", shape=[1]) 
elif 1:
    with tf.variable_scope("one"):
        a = tf.get_variable("v", shape=[1]) 
    with tf.variable_scope("one", reuse=True):
        b = tf.get_variable("v", shape=[1]) 
elif 1:
    with tf.variable_scope("one"):
        a = tf.get_variable("v", shape=[1]) 
        tf.get_variable_scope().reuse_variables()
        b = tf.get_variable("v", shape=[1]) 

assert(a is b)  #Assertion is true, they refer to the same object.

print(a.name)
print(b.name)

print(hex(id(a)))
print(hex(id(b)))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    print(sess.run([a, b]))

In all examples presented above, we shared parameters only because their names agreed, that is, because we opened a reusing variable scope with exactly the same string. In more complex cases, it might be useful to pass a VariableScope object rather than rely on getting the names right. To this end, variable scopes can be captured and used instead of names when opening a new variable scope.

https://chromium.googlesource.com/external/github.com/tensorflow/tensorflow/+/r0.7/tensorflow/g3doc/how_tos/variable_scope/index.md

# tf.variable_scope

In [1]:
import tensorflow as tf

with tf.variable_scope("foo") as foo_scope:
    v = tf.get_variable("v", [1])
    
with tf.variable_scope(foo_scope):
    w = tf.get_variable("w", [1])
    
with tf.variable_scope(foo_scope, reuse=True):
    v1 = tf.get_variable("v", [1])
    w1 = tf.get_variable("w", [1])
    
assert v1 == v
assert w1 == w

print(v.name)
print(w.name)
print(v1.name)
print(w1.name)

/Users/sungchul/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/sungchul/anaconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


foo/v:0
foo/w:0
foo/v:0
foo/w:0


In [1]:
import tensorflow as tf

with tf.variable_scope("foo") as foo_scope:    
    print(foo_scope.name) # foo
    
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        print(other_scope.name) # bar/baz

/Users/sungchul/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/sungchul/anaconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


foo
bar/baz


In [1]:
import tensorflow as tf

with tf.variable_scope("foo") as foo_scope:
    print(foo_scope.name)
    
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        with tf.variable_scope(foo_scope) as foo_scope2:
            print(foo_scope2.name) # foo"  

/Users/sungchul/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/sungchul/anaconda/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


foo
foo
